## Assessment Title: Integrated CA2

## Lecturer Name: David McQuaid & Muhammad Iqbal

## Student Full Name & Student ID: Deniz OZSOY UGUR - 2023195

### Link to Dataset

ProjectTweets.csv

### GitHub Repository: 

CCT-Dublin Link:

https://github.com/CCT-Dublin/adv-data-big-data-ft-ca2-denizozsoyugur

The link used since the beginning of the project:

https://github.com/denizozsoyugur/2023195_IntegratedCA2



## Inital Data Processing

In [1]:
#pip install statsmodels

In [2]:
#pip install pandas matplotlib

In [3]:
#pip install dash dash-core-components dash-html-components dash-table plotly

In [4]:
#pip install dash-bootstrap-components

In [5]:
#pip install plotly

In [6]:
#pip install numpy

In [7]:
#pip install pandas

In [8]:
#pip install pmdarima

In [9]:
#pip install transformers

In [10]:
#pip install --upgrade transformers

In [11]:
#pip install numpy==1.24.0

In [12]:
#pip install torch

In [13]:
#pip install mysql-connector-python

In [14]:
#pip install -U pandas==1.5.3

In [15]:
# Import libraries

import numpy as np
import pandas as pd
import plotly.express as px
import calendar
import re

from dateutil.parser import parse 
import matplotlib as mpl

import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
import mysql.connector
from pyspark import SparkConf
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import udf, col

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import mysql.connector
import pandas as pd
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
# warnings

warnings.filterwarnings('ignore')  

In [17]:
df = pd.read_csv('/home/hduser/Desktop/ProjectTweets.csv')

In [18]:
df.columns = ['indx','ids','date','flag','user','text']

In [19]:
df = df.iloc[:,1:]

In [20]:
df.head(5)

,ids,date,flag,user,text
0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [21]:
df.describe(include ='all')

,ids,date,flag,user,text
count,1.599999e+06,1599999,1599999,1599999,1599999
unique,NaN,774362,1,659775,1581465
top,NaN,Mon Jun 15 12:53:14 PDT 2009,NO_QUERY,lost_dog,isPlayer Has Died! Sorry
freq,NaN,20,1599999,549,210
mean,1.998818e+09,NaN,NaN,NaN,NaN
std,1.935757e+08,NaN,NaN,NaN,NaN
min,1.467811e+09,NaN,NaN,NaN,NaN
25%,1.956916e+09,NaN,NaN,NaN,NaN
50%,2.002102e+09,NaN,NaN,NaN,NaN
75%,2.177059e+09,NaN,NaN,NaN,NaN


In [22]:
df.iloc[:,4:] = df.iloc[:,4:].apply(lambda x: x.str.replace(r'[^A-Za-z0-9]+',''))

/tmp/ipykernel_15208/3213551197.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.iloc[:,4:] = df.iloc[:,4:].apply(lambda x: x.str.replace(r'[^A-Za-z0-9]+',''))


In [23]:
df.iloc[:,4:] = df.iloc[:,4:].apply(lambda x: x.str.replace('\\',''))

/tmp/ipykernel_15208/1778178343.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.iloc[:,4:] = df.iloc[:,4:].apply(lambda x: x.str.replace('\\',''))


In [24]:
df = df.drop_duplicates()

In [25]:
df.describe(include ='all')

,ids,date,flag,user,text
count,1.598314e+06,1598314,1598314,1598314,1598314
unique,NaN,774362,1,659775,1573475
top,NaN,Mon Jun 15 12:53:14 PDT 2009,NO_QUERY,lost_dog,Goodmorning
freq,NaN,20,1598314,549,234
mean,1.998872e+09,NaN,NaN,NaN,NaN
std,1.935909e+08,NaN,NaN,NaN,NaN
min,1.467811e+09,NaN,NaN,NaN,NaN
25%,1.956927e+09,NaN,NaN,NaN,NaN
50%,2.002114e+09,NaN,NaN,NaN,NaN
75%,2.177094e+09,NaN,NaN,NaN,NaN


In [26]:
# Create the spark Session
spark = SparkSession.builder.appName("sentiment_load").config("spark.driver.extraClassPath","/usr/local/spark/jars/mysql-connector-java-8.0.33.jar").getOrCreate()

24/05/09 02:40:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [27]:
# Create a user defined fucntion (UDF) to apply the sentiment analysis to the select text of the tweets df
def get_sentiment_score(text):
    result = sentiment_analyzer(text)
    score = result[0]["score"]
    return score

In [28]:
# Create a user defined fucntion (UDF) to create a new date column with an abbreviated value
def extract_and_concat(input_string):
    first_5 = input_string[:10]
    last_4 = input_string[-4:]
    return f"{first_5} {last_4}"

In [29]:
# Register the UDF's with the current spark session
spark.udf.register("get_sentiment_score", get_sentiment_score)
spark.udf.register("extract_and_concat", extract_and_concat)

<function __main__.extract_and_concat(input_string)>

In [30]:
# Set the MySQL connection settings
jdbc_url = "jdbc:mysql://localhost:3306/Tweets"
connection_properties = {
    "user":"root",
    "password":"password",
    "driver":"com.mysql.cj.jdbc.Driver"
}

In [34]:
# Apply the date abbreviation UDF to the date column, and create a new column "full_date", with the results
df['new_date'] = df['date'].apply(extract_and_concat)